In [183]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import plotly
from plotly.graph_objs import Scatter, Layout
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go

#plotly.offline.init_notebook_mode(connected=True)
%matplotlib inline

In [184]:
df = pd.read_csv('../for_images.csv', index_col=0)

In [185]:
df = df.drop(['date_diffs', 'dates_in_wiki', 'Wiki Content', 'Inaugural',
         'clean_wiki', 'clean_inaug'], axis=1)

In [186]:
dead_df = df.dropna(subset=['years_lived_after_pres'])

In [187]:
dead_df.columns

Index(['First Year of Presidency', 'President Name', 'Num Wiki References',
       'Words on Page', 'rank', 'birth_year', 'death_year', 'years_lived',
       'age_become_pres', 'years_lived_after_pres', 'profile_dates_mean',
       'profile_dates_std', 'presidency_length', 'cos_similarity_wiki_inaug',
       'more_than_10000_deaths', 'sim_rank'],
      dtype='object')

In [188]:
dead_df = dead_df.sort_values(by='First Year of Presidency')

In [93]:
#basic scatter, with horizontal orientation
trace1 = go.Scatter(x=dead_df['First Year of Presidency'], 
                    y=dead_df['profile_dates_mean'],
                   name = 'Mean Difference in Years',
                    mode='markers',
                    marker = dict(
                    size = 10,
                    color= 'rgba(20, 30, 200, .8)',
                    line= dict(width = 2)),
                    text = dead_df['President Name'],
                   hoverinfo= 'text')

trace2 = go.Bar(x=dead_df['First Year of Presidency'],
                y= dead_df['years_lived_after_pres'], 
                name = 'Years lived after Presidency',
                width=2.4,
                marker = dict(color='rgba(200,30,20,.8)'),
                opacity=0.32,
                 yaxis='y2',
               text = dead_df['years_lived_after_pres'],
               hoverinfo='text')

data = go.Data([trace1, trace2])

layout = go.Layout(title = 'Average Date on Wikipedia Page',
                   xaxis = dict(title = 'First year of Presidency'),
                   yaxis= dict(title = 'Difference between average year and first year',
                              titlefont=dict(color='rgba(20, 30, 200, .8)')),
                   yaxis2= dict(title= 'Years lived after Presidency', overlaying='y', side='right', 
                                titlefont=dict(color='rgba(200,30,20,.8)'))
                  )
#hovermode will put the y and x axis values into 

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [155]:
inaug = pd.read_csv('./inaug_sims.csv', index_col=0)
names = inaug.index.tolist()
names = [name.split()[-1] for name in names]
colors = cl.scales['9']['div']['RdYlBu']


trace = go.Heatmap(z=inaug.values.tolist(),
                  colorscale=[[0, colors[8]],
                             [0.10, colors[7]],
                             [0.15, colors[6]],
                             [0.20, colors[5]],
                             [0.25, colors[4]],
                             [0.30, colors[3]],
                             [0.35, colors[2]],
                             [0.40, colors[1]],
                             [1, colors[0]]],
                  colorbar = dict(tickmode='array',
                                  tickvals=[0.15, 0.95],
                                 ticktext = ['Less Similar', 'More Similar'],
                                 ticks= 'outside'))

layout = go.Layout(title = 'Similarities in Inaugural Addresses',
    yaxis= dict(ticktext = names , tickvals = np.arange(39)),
    xaxis=dict(ticktext = names , tickvals = np.arange(39)),
                   height=650,
                   width=650
                )

data = [trace]

fig = go.Figure(data=data, layout=layout, )
py.iplot(fig)

In [189]:
#reversing the order (so bigger is better)
dead_df['rank'] = np.abs(dead_df['rank'] - 43).values
dead_df = dead_df.sort_values(by='rank')

In [197]:
colors = cl.scales['3']['div']['PiYG']

In [207]:
trace = go.Scatter(x=dead_df['rank'], y=dead_df['Words on Page'],
                  mode='markers',
                  marker=dict(size= 16,
                              line= dict(width=2, color=(0,0,0,0)),
                              color=dead_df['more_than_10000_deaths'],
                             colorscale=[[0, colors[0]],
                                        [0, colors[0]],
                                        [1,colors[2]],
                                        [1,colors[2]]],
                             showscale=True,
                           colorbar = dict(tickmode='array',
                                      tickvals=[0.15, 0.95],
                                     ticktext = ['Fewer than 10,000 war deaths', 'More than 10,000 war deaths'],
                                 ticks= 'outside')),
                  text = dead_df['President Name'],
                  hoverinfo='text')

data = go.Data([trace])
layout = go.Layout(title='Determining Page Length',
                   xaxis=dict(title='CSPAN Rank'),
                  yaxis=dict(title='Words on Wikipedia page'))
fig = go.Figure(data=data, layout=layout)
plotly.iplot(fig)